In [3]:
import glob, os, re, json, pickle
import pandas as pd
from classifier import *
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

%reload_ext autoreload
%autoreload 2

# Neue Keywords

Mit den Bundestags Protokollen, den Manifesto Daten und den tweets kann man nochmal neue keywords generieren

In [2]:
stopwords = [x.strip() for x in open('stopwords.txt').readlines()[7:]]

label='all_tweets'
top_what=20
df_bundestag = get_bundestag_data()
df_manifesto = get_manifesto_data()
df_human = get_human_data()
text = pd.concat([df_bundestag['rede'],df_manifesto['text'],df_human['text']],axis=0)
fn = os.path.join(DATADIR, 'classifier-{}.pickle'.format(label))
clf = pickle.load(open(fn,'rb'))
labels_normalized = StandardScaler().fit_transform(clf.predict_proba(text))
vectorizer = clf.steps[0][1]
data_scaled = StandardScaler(with_mean=False).fit_transform(vectorizer.transform(text))
keywords = {}
for iclass, classname in enumerate(clf.steps[1][1].classes_):
    if classname not in ['undefined', 'ignored']:
        pattern = labels_normalized[:,iclass].T @ data_scaled
        idx2word = {idx: word for word, idx in vectorizer.vocabulary_.items()}
        words = pd.Series([idx2word[idx] for idx in pattern.argsort()[::-1]])
        exclude = words.isin(stopwords) | words.str.contains('\d+')
        keywords[classname] = words[~exclude].dropna()[:top_what].values
df_keywords = pd.DataFrame(keywords)
df_keywords.to_csv('keywords.csv', index=False)

Loading manifesto/manifesto-Germany.csv


/Users/felix/anaconda3/envs/pdds_1920/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.22.2.post1 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/felix/anaconda3/envs/pdds_1920/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.22.2.post1 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/felix/anaconda3/envs/pdds_1920/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SGDClassifier from version 0.22.2.post1 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/felix/anaconda3/envs/pdds_1920/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from 

In [4]:
df_keywords.iloc[:top_what,:]


,agriculture +,anti-growth economy +,anti-imperialism +,centralism +,constitution +,constitution -,controlled economy +,corporatism +,culture +,decentralism +,democracy +,economic goals,economic orthodoxy +,economic planning +,education +,education -,environmentalism +,europe +,europe -,foreign special +,foreign special -,free enterprise +,freedom/human rights +,gov-admin efficiency +,incentives +,infrastructure +,internationalism +,internationalism -,keynesian demand +,labour +,labour -,law and order +,market regulation +,marxist analysis +,middle class +,military +,military -,minority groups +,multiculturalism +,multiculturalism -,national way of life +,national way of life -,nationalization +,non economic groups +,peace +,political authority +,political corruption -,productivity +,protectionism +,protectionism -,social harmony +,social justice +,traditional morality +,traditional morality -,welfare +,welfare -
0,landwirtschaft,nachhaltige,machtpolitischer,bund,grundgesetz,grundgesetz,mindestlohn,arbeitgebern,kultur,kommunen,demokratie,arbeitsplätze,schulden,stadtumbau,bildung,hochschulen,klimaschutz,europäische,eu,russland,wiederaufflackern,eigenverantwortung,menschenrechte,bürokratie,unternehmen,forschung,nationen,globalisierungskritikerinnen,binnennachfrage,arbeit,drittelparität,kriminalität,verbraucher,neoliberalismus,mittelstand,bundeswehr,abrüstung,behinderungen,einwanderungsland,integration,land,npd,privatisierung,frauen,frieden,koalition,korruption,wachstum,freihandels,handelshemmnisse,engagement,frauen,familien,lebenspartnerschaften,versorgung,vorsorge
1,landwirte,nachhaltigkeit,interventionismus,betreuungsstandards,grundgesetzes,kinderrechte,gesetzlichen,sozialpartnerschaft,kunst,gemeinden,demokratische,beschäftigung,haushalte,ost,hochschulen,hochschulsanierungen,umwelt,eu,mitgliedstaaten,partnerschaft,raketenstationierungen,wettbewerb,freiheit,verwaltung,mittelstand,infrastruktur,vereinten,partnerstaaten,investitionen,beschäftigten,vorstandsentscheidungen,polizei,verbraucherinnen,kapitalismus,mitte,soldaten,atomwaffen,migrantinnen,integration,zuwanderung,sprache,rechtsextremismus,daseinsvorsorge,kinder,friedenspolitik,fraktion,abgeordnetenbestechung,wirtschaftswachstum,freihandelsabkommen,exportsubventionen,bürgerschaftliches,gerechtigkeit,familie,lebensgemeinschaften,grundsicherung,private
2,bauern,ökologische,machtinteressen,ländern,verfassung,gg,flächendeckenden,gewerkschaften,kulturelle,länder,bürgerinnen,wettbewerbsfähigkeit,konsolidierung,solidarpakt,schulen,trägerschaft,energien,europa,union,beziehungen,vereinnahmung,private,daten,vereinfachung,kleine,weiterbildung,internationalen,entwicklungsländer,öffentliche,arbeitnehmer,gewerkschaftsvertretern,gewalt,marktwirtschaft,herrschenden,rückgrat,nato,bundeswehr,opfer,vielfalt,erlernen,deutschlands,antisemitismus,hand,ältere,krisenprävention,genau,nebeneinkünfte,wirtschaftliches,transnationaler,doha,solidarität,teilhabe,eltern,cannabis,rente,eigenverantwortung
3,agrarpolitik,nachhaltiges,orientierende,bildungsstandards,werteordnung,absätze,vergleichsmiete,tarifautonomie,sport,bund,parlament,räume,neuverschuldung,west,studierenden,privatwirtschaftliches,natur,europäischen,europäischen,transatlantische,hegemonialer,staat,bürgerrechte,effizienz,existenzgründer,technologien,internationale,budgethilfen,lohnsteigerungen,arbeitsbedingungen,abfindungsregelungen,straftaten,wettbewerb,finanzkapitalismus,berufe,soldatinnen,rüstungskontrolle,migranten,einbürgerung,zuwanderern,deutsche,rassismus,eigentum,jugendliche,zivile,bundesregierung,bestechlichkeit,wohlstand,behandelter,abschottung,zusammenhalt,einkommen,kinder,paare,pflege,altersvorsorge
4,ernährungswirtschaft,nachhaltigen,punktesysteme,abschlussbezogene,forderndes,staatsziel,ortsüblichen,sozialpartnern,kulturpolitik,kommunalen,demokratischen,ländlichen,euro,programm,schule,sponsoring,emissionen,union,europäische,transatlantischen,stützpunkte,privatisierung,datenschutz,reform,wagniskapital,mobilität,entwicklungszusamm